In [1]:
import requests as rq  # запросы сети
import pandas as pd  # таблицы
from bs4 import BeautifulSoup  # парсинг
import re  # регулярные выражения
import pymorphy3  # морфологический анализатор

morph = pymorphy3.MorphAnalyzer()

dir = "D:\HSE\Pump_Dump\\"  # папка проекта

# список символов для удаления
with open(dir + "market\\clear_name.txt", encoding="utf8") as tmp_txt:
    clear_sym = tmp_txt.read().split("\n")

In [2]:
# нормализация значений
# строки для удаления из наименований
def clear_name(tmp_txt):
    for sym in clear_sym:
        tmp_txt = tmp_txt.lower()
        tmp_txt = tmp_txt.replace(sym, "")
        tmp_txt = tmp_txt.strip()
    return tmp_txt


# выборочные параметры акций во фрейме ММВБ
def moex_shares_ru():
    # считываем список ЦБ ММВБ
    shares_pd = pd.read_csv(dir + "market\\moex_cb.csv", sep=",", encoding="utf-8")
    # отбираем только акции
    shares_pd = shares_pd[(shares_pd["SUPERTYPE"] == "Акции")]
    # ИНН приводим к строке, удаляем .0, удаляем строки без ИНН
    shares_pd["INN"] = shares_pd["INN"].astype("string")
    shares_pd["INN"] = shares_pd["INN"].str.split(".", expand=True)[0]
    shares_pd.dropna(subset="INN", axis=0, inplace=True)
    # удаляем строки без тикеров
    shares_pd.dropna(subset="TRADE_CODE", axis=0, inplace=True)
    # Тикеры к нижнему регистру
    shares_pd["TRADE_CODE"] = shares_pd["TRADE_CODE"].str.lower()
    # чистим полные наименования
    for i in range(len(shares_pd)):
        shares_pd["EMITENT_FULL_NAME"].values[i] = clear_name(
            shares_pd.iloc[i]["EMITENT_FULL_NAME"]
        )
    return shares_pd[["INN", "TRADE_CODE", "EMITENT_FULL_NAME"]]


#
def normalize_name(full_name, short_name):
    norm_name=""
    # полное равно краткому (напр. Алроса) или краткое входит в полное (новабев групп и новабев)
    if full_name == short_name or short_name in full_name.split():
        # используем краткое
        norm_name = short_name
        # полное наименование состоит из одного слова и в нем нет гласных
    elif (
        len(full_name.split()) == 1
        and len(re.findall(r"[аеоуиёйюэ]", full_name.split()[0])) > 0
    ):
        # используем полное
        norm_name = full_name
    return norm_name

In [3]:
# актуальный файл ЦБ с ММВБ. Всегда перезаписываем, ежедневное обновление
def moex_allsecurities():
    page = rq.get("https://www.moex.com/ru/listing/securities-list-csv.aspx?type=1")
    f = open(dir+"market\\moex_cb.csv", "w", encoding="utf-8")  # открытие в режиме записи
    f.write(page.text)  # запись
    f.close()  # закрытие файла

moex_allsecurities()

# краткое наименование по тикеру
def moex_shortname_by_ticker(ticker):
    # запрос ММВБ по тикеру
    page = rq.get("https://iss.moex.com/iss/engines/stock/markets/shares/securities/"+ ticker+ "/securities.xml")
    raw_text = BeautifulSoup(page.text, features="xml")
    short_name = raw_text.find("row")["SHORTNAME"]
    short_name = clear_name(short_name)
    return short_name

In [4]:
# свой корпус акций
def my_shares_list():
    try:
        # загрузка файла
        df = pd.read_csv(
            dir + "market\\moex_my.csv", encoding="utf-8", sep="|", keep_default_na=False
        )
    except:
        cols = {
            "INN": [],
            "FULL_NAME": [],
            "TICKERS": [],
            "SHORT_NAME": [],
            "NORM_NAME": [],
            "MORPH_NAMES": [],
            "MY_NAMES": [],
        }
        df = pd.DataFrame(cols)
        df.to_csv(
            dir + "market\\moex_my.csv",
            index=False,
            encoding="utf-8",
            sep="|",
        )
    return df

In [ ]:
shares_pd = moex_shares_ru()
grouped = shares_pd.groupby(["INN"])
df = my_shares_list()
df["INN"] = df["INN"].astype("string")
for name, group in grouped:
    inn = name[0]
    # есть ли среди ИНН ММВБ новые? Если да, то получим краткое наименование со страницы эмитента (/ любую другую информацию)
    if inn not in list(df["INN"]):
        full_name = group["EMITENT_FULL_NAME"].values[0]
        tickers = ", ".join(group["TRADE_CODE"].values)
        ticker = group["TRADE_CODE"].values[0]
        short_name = moex_shortname_by_ticker(ticker)
        norm_name = normalize_name(full_name, short_name)
        if norm_name != "":
            # если мое наименование существует, то используем его для склонений
            new_names = set()
            voc_temp_norm = morph.parse(norm_name)[0].lexeme
            for s in voc_temp_norm:
                new_names.add(s.word)
            morph_names = ",".join(new_names)
        else:
            morph_names = ""
        print(inn, full_name, tickers, short_name, norm_name, morph_names, sep="|")

        # добавляем их во фрейм
        data = [
            {
                "INN": inn,
                "FULL_NAME": full_name,
                "TICKERS": tickers,
                "SHORT_NAME": short_name,
                "NORM_NAME": norm_name,
                "MORPH_NAMES": morph_names,
            }
        ]
        i_df = pd.DataFrame.from_dict(data)
        df = pd.concat([df, i_df], ignore_index=True)
        # сохранение в файл. Можно добавить проверку наличия файла, ИНН и заполнения MY_NAMES. После этого проводить апдейт файла
        df.to_csv(
            dir + "market\\moex_my.csv",
            index=False,
            encoding="utf-8",
            sep="|",
        )

In [ ]:
# загружаем итог и смотрим результат
df = pd.read_csv(
    dir + "market\\moex_my.csv", encoding="utf-8", sep="|", keep_default_na=False
)
df

,INN,FULL_NAME,TICKERS,SHORT_NAME,NORM_NAME,MORPH_NAMES,MY_NAMES
0,1215099739,тнс энерго марий эл,"misb,misbp",тнсэнмарэл,,,
1,1326192645,мордовская энергосбытовая,mrsb,мордэнсб,,,
2,1402047184,селигдар,selg,селигдар,селигдар,"селигдаром,селигдар,селигдара,селигдаров,селиг...",
3,1433000147,алроса,alrs,алроса,алроса,"алроса,алросе,алросом,алрос,алросу",
4,1435028701,якутскэнерго,"yken,ykenp",якутскэнрг,якутскэнерго,якутскэнерго,
...,...,...,...,...,...,...,...
189,9704168849,смарттехгрупп,carm,стг,смарттехгрупп,"смарттехгруппой,смарттехгруппами,смарттехгрупп...",
190,9718077239,группа позитив,posi,позитив,позитив,"позитивов,позитивом,позитив,позитивы,позитиве,...",
191,9731004688,группа компаний самолет,smlt,самолет,самолет,"самолёта,самолёте,самолёту,самолётом,самолётам...",
192,9731078633,центр генетики и репродуктивной медицины генетико,geco,генетико,генетико,генетико,
